É preciso criar mais imagens com sample de modo a equilibrar o dataset 

Devo utilizar um modelo em que não sejam necessárias imagens com RGB?

Perguntar a David: precisamos de mais data ou de um modelo mais simples?

Aumentar dataset e depois retirar aleatoriamente imagens com label ambiente

fazer modelo simples com 3 layers

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python import train
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os,sys

2023-08-29 11:28:09.583948: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-29 11:28:09.772528: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-29 11:28:09.773777: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 11:28:10.984130: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df_subImages = pd.read_pickle('subImages')

In [3]:
df_subImages.head(5)

,subImages,label
0,"[[0.12156863, 0.12156863, 0.12156863, 0.121568...",0
1,"[[0.15686275, 0.15686275, 0.15686275, 0.156862...",0
2,"[[0.1882353, 0.1882353, 0.18039216, 0.18431373...",0
3,"[[0.18039216, 0.1764706, 0.1882353, 0.19607843...",0
4,"[[0.14901961, 0.14901961, 0.13333334, 0.113725...",0


In [4]:
import tensorflow.keras as keras
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, MaxPool2D
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator                    

In [5]:
X = np.stack(df_subImages['subImages'].to_numpy())
y = df_subImages['label'].to_numpy()
# y = np.where(y == 'sample',1,0)

X = X[..., np.newaxis]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, stratify=y, random_state=42)

print(X.shape)
print(X_train.shape)
print(X_test.shape)

(3136, 96, 128, 1)
(2101, 96, 128, 1)
(1035, 96, 128, 1)


# Data Augmentation

In [6]:
def plotImages(images_arr):
    fig, axes = plt.subplots(2,5,figsize=(120,120))
    fig.subplots_adjust(wspace=0, hspace=0)
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes.ravel()):
        ax.imshow(img, cmap='gray')
        ax.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
    plt.tight_layout()
    plt.show()

In [7]:
training_datagen = ImageDataGenerator(
      rescale = 1./255,
      # rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

TODO: Ser mais folgado com o que é considerado sample ou não

In [8]:
it = training_datagen.flow(X_train, y_train, batch_size=10)

# print(next(it)[0].shape)
# print(next(it)[1].shape)

aug_images = [next(it) for i in range(10)]

# print(aug_images[0][0].shape)

print(aug_images[0][1])

plotImages(aug_images[0][0])

[0 0 0 0 0 0 0 1 0 0]


# Neural Network

## Hand made model ##

In [9]:
X_train[0].shape

(96, 128, 1)

In [10]:
data_augmentation = tf.keras.Sequential([
  layers.Rescaling(1./255),
  layers.RandomFlip("vertical", seed=42),
  layers.RandomZoom(0.2, seed=42),
  layers.RandomBrightness(0.2, seed=42),
  layers.RandomContrast(0.2, seed=42)
])

training_datagen = ImageDataGenerator(
      rescale = 1./255,
      # rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [11]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(y_train), y= y_train)

In [ ]:
model = keras.models.Model()

input_layer = keras.Input(shape=(96, 128,1))
x = data_augmentation(input_layer)

x = Conv2D(filters=32, kernel_size=(5,5), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2,2), strides=2)(x)
x = Conv2D(filters=64, kernel_size=(5,5), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2,2), strides=2)(x)
x = Conv2D(filters=128, kernel_size=(5,5), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2,2), strides=2)(x)
x = Flatten()(x)
x = Dense(128)(x)
x = Dense(2, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=x)


# print(model.summary())

model.compile(optimizer="SGD", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics="acc")

history = model.fit(x=X_train, y=y_train, batch_size=256, epochs=5, use_multiprocessing = True, validation_split=0.2,class_weight=dict(enumerate(class_weights)),shuffle=True)

In [37]:
model.evaluate(x=X_test, y=y_test)

33/33 [==============================] - 2s 46ms/step - loss: 0.6780 - acc: 0.9652


[0.6780499219894409, 0.9652174115180969]

In [28]:
print(model.predict(X_test))

33/33 [==============================] - 2s 43ms/step
[[0.52367646 0.47632352]
 [0.5236383  0.47636175]
 [0.5236129  0.47638708]
 ...
 [0.52362245 0.47637755]
 [0.523639   0.47636092]
 [0.5237135  0.4762865 ]]


## VGG16

<img src="1 Q_bg1E3trWcjdk9_jnVGwg.png" alt="Alternative text" />

Converts a grayscale image to an rgb image. Also converts into a tensor so it can be fed to the network

In [15]:
print(dadasd)

NameError: name 'dadasd' is not defined

In [13]:
def process_img(img):
    img = img[..., np.newaxis]
    img_tensor = tf.convert_to_tensor(img)
    converted = tf.image.grayscale_to_rgb(
        img_tensor,
        name=None
    )
    return converted

df_subImages['subImages'] =  df_subImages['subImages'].apply(process_img)

In [14]:
base_model = VGG16(weights='imagenet', include_top=False, pooling = 'max', input_shape=(96, 128, 3))
base_model.trainable = False


flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(512)
dense_layer_2 = layers.Dense(2)
prediction_layer = layers.Softmax()

model = Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

base_model.summary()

X_train = preprocess_input(X_train) 
X_test = preprocess_input(X_test) 

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 128, 3)]      0         
                                                                 
 block1_conv1 (Conv2D)       (None, 96, 128, 64)       1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 96, 128, 64)       36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 48, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 48, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 48, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 24, 32, 128)       0     

IndexError: index 1 is out of bounds for axis 3 with size 1

In [ ]:
model.fit(X_train, y_train, epochs=2, batch_size=16, validation_split=0.2)

Epoch 1/2


2023-08-28 10:32:03.736474: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 247726080 exceeds 10% of free system memory.
/home/dan/anaconda3/envs/new_env/lib/python3.11/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(16, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)
2023-08-28 10:32:04.614922: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 50331648 exceeds 10% of free system memory.
2023-08-28 10:32:04.643543: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 50331648 exceeds 10% of free system memory.
2023-08-28 10:32:04.699166: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25165824 exceeds 10% of free system memory.
2023-08-28 10:32:04.722386: W tensorflow/tsl/framework/cpu_allocator_impl.cc

105/105 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.9696

/home/dan/anaconda3/envs/new_env/lib/python3.11/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


105/105 [==============================] - 55s 514ms/step - loss: 0.0000e+00 - accuracy: 0.9696 - val_loss: 0.0000e+00 - val_accuracy: 0.9501
Epoch 2/2
105/105 [==============================] - 59s 567ms/step - loss: 0.0000e+00 - accuracy: 0.9696 - val_loss: 0.0000e+00 - val_accuracy: 0.9501


In [ ]:
model.evaluate(x=X_test, y=y_test)

33/33 [==============================] - 27s 819ms/step - loss: 0.0000e+00 - accuracy: 0.9652


[0.0, 0.9652174115180969]

In [ ]:
predictions = model.predict(X_test)

33/33 [==============================] - 29s 870ms/step


In [ ]:
predictions

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python import train
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, ReLU, Input, Softmax, Flatten, Conv2D, MaxPool2D, Dropout, Conv2DTranspose, Reshape
from tensorflow.keras.optimizers import Adam, RMSprop, Adagrad




encoder_input = Input(shape=(768,1024,1), name="img") #the 1 is the number of channels, if the images were RGB there would be a 3
conv_1 = Conv2D(16,kernel_size=(3,4),strides=(3,4),padding="same", activation="relu")(encoder_input)
maxpool_1 = MaxPool2D(pool_size=(2,2),strides=(1, 1),padding="valid")(conv_1)

conv_2 = Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="same", activation="relu")(maxpool_1)
maxpool_2 = MaxPool2D(pool_size=(2,2),strides=(2, 2),padding="valid")(conv_2)

conv_3 = Conv2D(64,kernel_size=(3,3),strides=(1,1),padding="same", activation="relu")(maxpool_2)
maxpool_3 = MaxPool2D(pool_size=(2,2),strides=(2, 2),padding="valid")(conv_3)

conv_4 = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding="same", activation="relu")(maxpool_3)
maxpool_4 = MaxPool2D(pool_size=(2,2),strides=(2, 2),padding="valid")(conv_4)

conv_5 = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding="same", activation="relu")(maxpool_4)




maxpool_5 = MaxPool2D(pool_size=(2,2),strides=(2, 2),padding="valid")(conv_5)

conv_6 = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding="same", activation="relu")(maxpool_5)
maxpool_6 = MaxPool2D(pool_size=(2,2),strides=(2, 2),padding="valid")(conv_6)


conv_7 = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding="same", activation="relu")(maxpool_6)
conv_8 = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding="same", activation="relu")(conv_7)





conv_9 = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding="same", activation="relu")(conv_8)
conv_10 = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding="same", activation="relu")(conv_9)
#YOLO









encoder = keras.Model(encoder_input, maxpool_4, name="encoder")

#decoder_input = keras.layers.Dense(64, activation="relu")(encoder_output)
#x = keras.layers.Dense(784, activation="relu")(decoder_input)
#decoder_output = keras.layers.Reshape((28,28,1))(x)

#opt = keras.optimizers.Adam(lr=0.001)

#autoencoder =  keras.Model(encoder_input, decoder_output, name="autoencoder")

encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img (InputLayer)            [(None, 768, 1024, 1)]    0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 16)      208       
                                                                 
 max_pooling2d (MaxPooling2  (None, 255, 255, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 255, 255, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 127, 127, 64)      1849